In [1]:
import pandas as pd
import numpy as np
import acquire
import matplotlib.pyplot as plt
import nltk
import unicodedata
import re
import env
import acquire
# import prepare
from wordcloud import WordCloud
import nltk.sentiment
pd.set_option('display.max_colwidth', -1)
from time import strftime
sia = nltk.sentiment.SentimentIntensityAnalyzer()

In [2]:
df = acquire.get_df()

In [3]:
df.head()

,episode_id,episode_order,speaker,utterance,program,title,episode_date,is_host
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling Magazine)","It's a 2,200-mile race. To give some sense of perspective, that's roughly the distance between Washington, D.C. and Las Vegas. They do it over the course of three weeks at very fast speeds. But incredibly, oftentimes the distance between first and second is somewhere between and one and three minutes.",Talk of the Nation,How To Watch The Tour De France,2010-07-12,False
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling Magazine)","So for a top competitor like Lance to try to make up that much time -he's now 13 minutes, 26 seconds behind the current race leader, Cadel Evans of Australia. And even Lance said yesterday that for him, the -any chance of winning the tour has gone out the window. He still does have a teammate on his team, RadioShack team, American Levi Leipheimer currently in eighth place, two minutes, 14 seconds back. And Lance is going to do what he can to help Leipheimer do well.",Talk of the Nation,How To Watch The Tour De France,2010-07-12,False
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star, one equivalent to Lance Armstrong and the other team - the rest of the teammates project him, do what they can to help him succeed?",Talk of the Nation,How To Watch The Tour De France,2010-07-12,True
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling Magazine)","That's right. Each team has nine riders. And what you have is basically the team leader, sort of your queen bee, and then eight worker bees working to keep that queen safe and happy for when the decisive moments of the race come. In cycling, a lot of the energy goes toward actually battling wind resistance. So if you're riding behind another rider, you can be using up to 30 percent less energy because you're in that slipstream of the rider who's battling the wind. So team riders will ride on the front, protect their leader. You'll often see a team member drift back to the team car to fetch water bottles to keep their leader well hydrated.",Talk of the Nation,How To Watch The Tour De France,2010-07-12,False
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car racing, right?",Talk of the Nation,How To Watch The Tour De France,2010-07-12,True


In [4]:
df.shape

(3199858, 8)

In [5]:
df= df.dropna()
df= df.drop_duplicates()

In [6]:
df.shape

(3186483, 8)

In [7]:
df.episode_date.min(), df.episode_date.max()

('1999-01-08', '2019-10-12')

In [8]:
df.dtypes

episode_id       int64 
episode_order    int64 
speaker          object
utterance        object
program          object
title            object
episode_date     object
is_host          bool  
dtype: object

In [9]:
df.program.value_counts()

All Things Considered       965976
Talk of the Nation          943016
Morning Edition             633786
News & Notes                183690
Day to Day                  173702
Weekend Edition Sunday      166596
Weekend Edition Saturday    119717
Name: program, dtype: int64

In [10]:
df.speaker.value_counts()

NEAL CONAN, host                                                                                                                                            204935
_NO_SPEAKER                                                                                                                                                 183707
IRA FLATOW, host                                                                                                                                            58172 
NEAL CONAN, HOST                                                                                                                                            43469 
FARAI CHIDEYA, host                                                                                                                                         36462 
                                                                                                                                                            ...   
President Obama has an

In [11]:
df = df[df.speaker!='_NO_SPEAKER']

In [12]:
df.head()

,episode_id,episode_order,speaker,utterance,program,title,episode_date,is_host
0,57264,9,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling Magazine)","It's a 2,200-mile race. To give some sense of perspective, that's roughly the distance between Washington, D.C. and Las Vegas. They do it over the course of three weeks at very fast speeds. But incredibly, oftentimes the distance between first and second is somewhere between and one and three minutes.",Talk of the Nation,How To Watch The Tour De France,2010-07-12,False
1,57264,10,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling Magazine)","So for a top competitor like Lance to try to make up that much time -he's now 13 minutes, 26 seconds behind the current race leader, Cadel Evans of Australia. And even Lance said yesterday that for him, the -any chance of winning the tour has gone out the window. He still does have a teammate on his team, RadioShack team, American Levi Leipheimer currently in eighth place, two minutes, 14 seconds back. And Lance is going to do what he can to help Leipheimer do well.",Talk of the Nation,How To Watch The Tour De France,2010-07-12,False
2,57264,11,"NEAL CONAN, host","So in every team, presumably there's one star, one equivalent to Lance Armstrong and the other team - the rest of the teammates project him, do what they can to help him succeed?",Talk of the Nation,How To Watch The Tour De France,2010-07-12,True
3,57264,12,"Ms. LOREN MOONEY (Editor-in-Chief, Bicycling Magazine)","That's right. Each team has nine riders. And what you have is basically the team leader, sort of your queen bee, and then eight worker bees working to keep that queen safe and happy for when the decisive moments of the race come. In cycling, a lot of the energy goes toward actually battling wind resistance. So if you're riding behind another rider, you can be using up to 30 percent less energy because you're in that slipstream of the rider who's battling the wind. So team riders will ride on the front, protect their leader. You'll often see a team member drift back to the team car to fetch water bottles to keep their leader well hydrated.",Talk of the Nation,How To Watch The Tour De France,2010-07-12,False
4,57264,13,"NEAL CONAN, host","So slipstream, this is like drafting in car racing, right?",Talk of the Nation,How To Watch The Tour De France,2010-07-12,True


In [13]:
df.shape

(3002776, 8)

In [14]:
df.episode_id.value_counts()

35108    512
73336    475
57481    459
19633    453
77896    419
        ... 
33474    1  
30384    1  
80379    1  
62026    1  
88152    1  
Name: episode_id, Length: 104896, dtype: int64

In [15]:
df[df.episode_id==19633].sort_values(by='episode_order')

,episode_id,episode_order,speaker,utterance,program,title,episode_date,is_host
1281018,19633,1,"IRA FLATOW, HOST","This is SCIENCE FRIDAY. I am Ira Flatow. It's back by popular request. OK, maybe that's not true: It's back because we enjoyed doing it so much last year. Ladies and gentlemen, it's time for our annual April Fools' science comedy geek-a-thon we call Two Scientists Walk Into a Bar.",Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False
1281019,19633,2,"IRA FLATOW, HOST","The question is simple: Can jokes about chemists marooned on an island or geneticists changing a light bulb really be funny? This hour we're going to attempt to prove that science is a laughing matter, at least some of the time. We have a humorist, a comedian and our very own Flora Lichtman, not in a bar or an island but in a studio to test the hypothesis that science is to comedy what Stephen Hawking is to black holes.",Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False
1281020,19633,3,"IRA FLATOW, HOST","And during the week, we asked for your help on our Facebook page. We asked you to tell us your favorite jokes, your science jokes, good ones, the not-so-good ones. Many of you contributed your favorite jokes, and some of them are even funny. We'll share them with you, plus some of our favorite scientists submitted their wit and humor, and we'll be sharing some of that with you this hour, too.",Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False
1281021,19633,4,"IRA FLATOW, HOST","So it's not too late if you still have a science funny you'd like to share. Give us a call, our number 1-800-989-8255, that's 1-800-989-TALK. And if you're on Twitter, you can tweet us a joke, and write @scifri, the @ sign followed by S-C-I-F-R-I, and don't know if you can do that in 140 characters, a joke? Try to get the joke in there. See, you give it a try.",Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False
1281022,19633,5,"IRA FLATOW, HOST","You can also go to our Facebook or facebook/scifri and join in our conversation, but just remember one thing. Here's the only rule that we have. It has to be a G-rated joke. I don't want to have to - where's the dump button? I don't want to have to push that dump button if it gets a little out of hand. So let's keep it cordial.",Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False
...,...,...,...,...,...,...,...,...
1281556,19633,564,"IRA FLATOW, HOST","Are you going to be appearing anywhere, Brian? We're running out of time. Are you going to appear anywhere doing standup that we could tell people about?",Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False
1281557,19633,565,BRIAN MALOW,"Well, right now, I'm going to be participating - I'm going to do a show at the Philadelphia Science Festival coming up on April 24 for the Chemical Heritage Foundation. And then at the end of April, the USA Science and Engineering Festival, I'm doing many events all throughout it on various stages and along with the National Academies.",Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False
1281558,19633,566,"IRA FLATOW, HOST","Yeah. Well, that's a great venue. And, Steve, you're just going to be hanging out at...",Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False
1281559,19633,567,STEVE MIRSKY,I'm going home with you tonight.,Talk of the Nation,No Joke: Science Is A Laughing Matter,2012-03-30,False


In [16]:
df['sentiment'] = df.utterance.apply(lambda doc: sia.polarity_scores(doc)['compound'])



KeyboardInterrupt: 

In [ ]:
df.head()